In [1]:
import pandas as pd

import random
import re
import pickle
import nltk 
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dpal2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dpal2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Load pre-trained models and data
import pickle
classifier = pickle.load(open('classifier.pickle', 'rb'))
vectorizer = pickle.load(open('TfidfModel.pickle', 'rb'))
# X and y are training data, may not be needed for inference

c:\Coding\Major Project\Fake-Product-Review-Monitoring-master\.venv\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\Coding\Major Project\Fake-Product-Review-Monitoring-master\.venv\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\Coding\Major Project\Fake-Product-Review-Monitoring-master\.venv\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.1 when using version 0.22.2.post1. Thi

In [3]:
dataset = pd.read_csv("reviews.csv",sep="\t")
dataset.drop(columns="Unnamed: 0", inplace=True, errors="ignore")
dataset

c:\Coding\Major Project\Fake-Product-Review-Monitoring-master\.venv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,timestamp,IP Address
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,82850235,BlackVue DR600GW-PMP,Mobile_Electronics,5,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,1.440988e+09,193.93.167.87
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,82850235,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0.0,1.0,N,Y,five star,it's great,2015-08-31,1.441002e+09,193.93.167.87
2,US,51469641,R2Y0MM9YE6OP3P,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5,0.0,0.0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31,1.440959e+09,193.93.167.87
3,US,4332923,RRB9C05HDOD4O,B00QUFTPV4,82850235,abcGoodefg® FBI Covert Acoustic Tube Earpiece ...,Mobile_Electronics,4,0.0,0.0,N,Y,Work very well,Work very well,2015-08-31,1.441015e+09,193.93.167.87
4,US,44855305,R26I2RI1GFV8QG,B0067XVNTG,563475445,Generic Car Dashboard Video Camera Vehicle Vid...,Mobile_Electronics,2,0.0,0.0,N,Y,Cameras has battery issues,"Be careful with these products, I have bought ...",2015-08-31,1.440973e+09,205.10.168.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88573,US,38978459,R2CDRVDUKB5Z9P,B00005OTZQ,554527960,Royal SE 2800 Hand-Held Spot Cleaner,Mobile_Electronics,5,23.0,27.0,N,N,The Cat Barf is Gone!,"I've been looking for a while for the \\""purr\...",2002-04-03,1.017782e+09,201.30.21.211
88574,US,51697602,R1DVLTZFXXOX9,B00005OTZQ,554527960,Royal SE 2800 Hand-Held Spot Cleaner,Mobile_Electronics,5,10.0,11.0,N,N,Well worth [it],We live in an apartment with hardwood floors a...,2002-03-05,1.015341e+09,216.59.233.79
88575,US,50891675,R1GHKKZMHAHQC1,B00005OTZQ,554527960,Royal SE 2800 Hand-Held Spot Cleaner,Mobile_Electronics,5,13.0,13.0,N,N,Its a Time saver!!!,I received this for a birthday present and Its...,2002-02-19,1.014068e+09,209.193.182.91
88576,US,38528439,R227G6T5B26DVC,B00005OTZQ,554527960,Royal SE 2800 Hand-Held Spot Cleaner,Mobile_Electronics,5,58.0,61.0,N,N,Got Stains?,I got the Dirt Devil Spot Scrubber for a gift ...,2002-01-06,1.010282e+09,205.32.108.104


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import nltk
import re
from nltk.corpus import stopwords 
import collections
from nltk.corpus import wordnet
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
dataset.set_index("review_id",inplace=True)

In [6]:
# Latent symantic analysis
# it will analyse all reviews and determine all reviews belong to the same concept
def LSA(text):
    #text is list of reviews of same product
    
    
    # Created TF-IDF Model
    vectorizer = TfidfVectorizer(stop_words="english")
    X = vectorizer.fit_transform(text)

    # Skip LSA if there is no meaningful variance
    if X.shape[0] < 2 or X.shape[1] < 2 or X.nnz == 0:
        return [0.0] * len(text)
    
    # Created SVD(Singular Value Decomposition)
    # suppress RuntimeWarning caused by zero variance matrices
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", RuntimeWarning)
        lsa = TruncatedSVD(n_components = 1, n_iter = 100, random_state=42)
        lsa.fit(X)
    
    terms = vectorizer.get_feature_names_out()

    concept_words={}

    for j,comp in enumerate(lsa.components_):
        componentTerms = zip(terms,comp)
        sortedTerms = sorted(componentTerms,key=lambda x:x[1],reverse=True)
        sortedTerms = sortedTerms[:10]
        concept_words[str(j)] = sortedTerms
     
    sentence_scores = []
    for key in concept_words.keys():
        for sentence in text:
            words = nltk.word_tokenize(sentence)
            scores = 0
            for word in words:
                for word_with_scores in concept_words[key]:
                    if word == word_with_scores[0]:
                        scores += word_with_scores[1]
            sentence_scores.append(scores)
    return sentence_scores


In [7]:
product_df = dataset.groupby("product_id")
#grouping dataset by product_id

unique_products = dataset["product_id"].unique()
#stores list of all product id

no_products = len(unique_products.tolist())
#no. of products

remove_reviews = []
#store review_id that are fake 
    
for i in range(no_products):
    #iterate through all product reviews 
    
    df = product_df.get_group(unique_products[i])
    #dataframe of a single product
    
    unique_reviews = df.index.tolist()
    #list of review_id of reviews of same product
    
    no_reviews = len(unique_reviews)   
    #no. of reviews of same product
    
    count = no_reviews 
    #count is no. of reviews that can be analysed
    
    reviews = []
    #list of review texts
    
    review_id = []
    #list of review texts
    
    for j in range(no_reviews):
        #iterate through all reviews
        
        text = str(df.loc[unique_reviews[j]]["review_body"])
        # text of a review 
        
        #cleaning the text
        text = re.sub(r"\W"," ",text)             
        text = re.sub(r"\d"," ",text)             
        text = re.sub(r"\s+[a-z]\s+"," ",text)    
        text = re.sub(r"^[a-z]\s+"," ",text)    
        text = re.sub(r"\s+[a-z]$"," ",text)    
        text = re.sub(r"\s+"," ",text)
        
        words = nltk.word_tokenize(text)
        #text into word list
        
        if(len(words)==1):
        #if only one word in text review
            
            if(len(text) <=1 or not wordnet.synsets(text) ):
            #if word is having only one character or invalid english word
                
                remove_reviews.append(unique_reviews[j])
                #append this review as fake
                
                count-=1
                #review left to be analysed will be decrease by 1
                
                continue
                #check for next review
                '''
            elif(len(words[0])<=1):
                remove_reviews.append(unique_reviews[j])
                count-=1
                continue
                '''
        elif(len(words)==0):
        #if no words
            
            remove_reviews.append(unique_reviews[j])
            #append this review as fake
            
            count-=1
            #review left to be analysed will be decrease by 1
            
            continue
            #check for next review
        
        review_id.append(unique_reviews[j])
        #valid: append review_id to review_id list for analysis
        
        reviews.append(text)
        #valid: append review_body to reviews list for analysis
        
    ###########################################################################################
    if(count<=0):
        #if no reviews left to analyse 

        continue
        #check for next
        
    if(count==1): 
        #if only one review is left to analyse
        
        #check concept between product title and the review
        
        text = [text,str(df.loc[review_id[0]]["product_title"])] 
        #add product_title and review to the list 
        
        sentence_scores = LSA(text) 
        #call LSA method to get the score
        
        if(sentence_scores[0]==0): 
        #if review score is 0, then it's fake
            remove_reviews.append(review_id[0])
        continue
    
    #list of scores of reviews of same product
    sentence_scores = LSA(reviews)
            
    for j in range(len(sentence_scores)):
        #iterating through all the scores
        
        if(sentence_scores[j]==0.00):
            # if score is 0, it's fake
            remove_reviews.append(review_id[j])

    # Content similarity check
    tfidf_vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = tfidf_vectorizer.fit_transform(reviews)
    for i in range(len(reviews)):
        sim_list = cosine_similarity(tfidf_matrix[i:i+1], tfidf_matrix)[0]
        for k in range(len(sim_list)):
            if k != i and sim_list[k] > 0.6:
                if review_id[k] not in remove_reviews:
                    remove_reviews.append(review_id[k])

# Output the results
print(f"Total fake reviews detected: {len(remove_reviews)}")
print("Fake review IDs:", remove_reviews)

# Save to pickle
with open('fake_reviews.pkl', 'wb') as f:
    pickle.dump(remove_reviews, f)
print("Results saved to fake_reviews.pkl")

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\dpal2/nltk_data'
    - 'c:\\Coding\\Major Project\\Fake-Product-Review-Monitoring-master\\.venv\\nltk_data'
    - 'c:\\Coding\\Major Project\\Fake-Product-Review-Monitoring-master\\.venv\\share\\nltk_data'
    - 'c:\\Coding\\Major Project\\Fake-Product-Review-Monitoring-master\\.venv\\lib\\nltk_data'
    - 'C:\\Users\\dpal2\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************
